In [9]:
def scan_line(s):
    result = {}
    for kv in s.split():
        k, v = kv.split(":")
        result[k] = v
    return result
scan_line("hgt:177cm pid:173cm")

{'hgt': '177cm', 'pid': '173cm'}

In [15]:
passports = []
passport = {}
with open("i.txt",'r') as f:
    for l in f:
        if l == "\n":
            passports.append(passport)
            passport = {}
        else:
            passport.update(scan_line(l))
passports[0]

{'eyr': '2033',
 'hgt': '177cm',
 'pid': '173cm',
 'ecl': 'utc',
 'byr': '2029',
 'hcl': '#efcc98',
 'iyr': '2023'}

In [18]:
import pandas as pd
df = pd.DataFrame(passports)
df.head()

,eyr,hgt,pid,ecl,byr,hcl,iyr,cid
0,2033,177cm,173cm,utc,2029,#efcc98,2023,NaN
1,2026,155cm,337605855,grn,1952,#866857,2017,249
2,2029,NaN,953198122,blu,NaN,#888785,2011,242
3,2030,173cm,112086592,amb,2011,#341e13,2012,341
4,2023,163cm,790332032,brn,1969,#623a2f,2019,NaN


In [126]:
valid_cols = ["eyr", "hgt", "pid", "ecl", "byr", "hcl", "iyr"]
df2=df[valid_cols].dropna()
df2.shape[0]

219

In [130]:
import re
col_regex = re.compile(r'^#(?:[0-9a-fA-F]{6})$')
yr_regex = re.compile(r'^\d{4}$')
pid_regex = re.compile(r'^\d{9}$')

ecl_list = ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]

def check_yr(input, earliest, latest):
    if yr_regex.match(input):
        n = int(input)
        return n >= earliest and n <= latest
    return False

def check_byr(input):
    return check_yr(input, 1920, 2002)

def check_iyr(input):
    return check_yr(input, 2010, 2020)

def check_eyr(input):
    return check_yr(input, 2020, 2030)

def check_hgt(input):
    unit = input[-2:]
    hgt = input[:-2]
    if not hgt.isdigit():
        return False
    hgt=int(hgt)
    if unit=="cm":
        return hgt >= 150 and hgt <= 193
    if unit=="in":
        return hgt >= 59 and hgt <= 76
    return False

def check_col(input):
    return bool(col_regex.match(input))

def check_ecl(input):
    return (input in ecl_list)

def check_pid(input):
    return bool(pid_regex.match(input)) 

    
    

In [131]:
df2.head()

,eyr,hgt,pid,ecl,byr,hcl,iyr
0,2033,177cm,173cm,utc,2029,#efcc98,2023
1,2026,155cm,337605855,grn,1952,#866857,2017
3,2030,173cm,112086592,amb,2011,#341e13,2012
4,2023,163cm,790332032,brn,1969,#623a2f,2019
5,2023,171cm,890112986,hzl,1920,#b6652a,2017


In [132]:
df2.apply(lambda row: check_byr(row['byr']) 
                     and check_iyr(row["iyr"]) 
                     and check_eyr(row["eyr"]) 
                     and check_hgt(row["hgt"]) 
                     and check_col(row["hcl"]) 
                     and check_ecl(row["ecl"]) 
                     and check_pid(row["pid"])
                     , axis=1).sum()

127